In [7]:
from openpyxl import load_workbook
import random

def informacoes_iniciais(escola):
  meses= ("","Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro")
  cnpj_buscar = str(input("Digite o CNPJ da escola que deseja bucar(sem . ou /, ex:123456789) : "))
  nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola = buscar_escola(escola,cnpj_buscar)
  print(f"\n este cnpj corresponde a escola {nome_escola}\n")
  dia = str(input("Digite a data dos orçamentos (dia): "))
  mes = int(input("Digite a data dos orçamentos (mês): "))
  if mes<1 or mes>12:
    print("o numero do mês está errado")
    exit(0)
  mes_digitado = meses[mes]
  ano = str(input("Digite a data dos orçamentos (ano): "))
  NF = str(input("Digite o número da nota fiscal correspondente : "))
  return nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola,dia,mes,mes_digitado,ano,NF

def buscar_escola(escola,buscador):
  for row in range(2,escola.max_row + 1):
    celula_buscar = escola.cell(row=row,column = 1).value
    if celula_buscar == buscador:
      nome_escola = escola.cell(row=row,column = 2).value
      cnpj_escola = escola.cell(row=row,column = 3).value
      cep_escola = escola.cell(row=row,column = 4).value
      cidade_escola = escola.cell(row=row,column = 5).value
      endereco_escola = escola.cell(row=row,column = 6).value
      return nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola
  print("Escola não encontrada")
  exit(0)

def aplicar_informacoes(pagina_escolas,nce,paper,grafite):
  nome_escola,cnpj_escola,cep_escola,cidade_escola,endereco_escola,dia,mes,mes_digitado,ano,NF =informacoes_iniciais(pagina_escolas)
  #nce
  pagina_nce.cell(row=6,column=1).value = f"NOSSA SENHORA DA GLÓRIA/SE {dia} DE {mes_digitado.upper()} DE {ano}"
  pagina_nce.cell(row=8,column=1).value = nome_escola
  pagina_nce.cell(row=9,column=1).value = endereco_escola
  pagina_nce.cell(row=10,column=1).value = f"CEP {cep_escola} - {cidade_escola}"
  pagina_nce.cell(row=11,column=1).value = f"CNPJ: {cnpj_escola}"
  #paper&co
  pagina_paper.cell(row=9,column=1).value = f"NOSSA SENHORA DAS DORES/SE, {dia} DE {mes_digitado.upper()} DE {ano}"
  pagina_paper.cell(row=11,column=1).value = f"{nome_escola}      /      CNPJ- {cnpj_escola}"
  pagina_paper.cell(row=12,column=1).value = endereco_escola
  pagina_paper.cell(row=13,column=1).value = f"CEP {cep_escola}   -   {cidade_escola}"
  #grafite
  pagina_grafite.cell(row=7,column=2).value = nome_escola.title()
  pagina_grafite.cell(row=7,column=5).value = cnpj_escola
  pagina_grafite.cell(row=8,column=2).value = endereco_escola.title()
  pagina_grafite.cell(row=9,column=2).value = cidade_escola.title()
  pagina_grafite.cell(row=10,column=2).value = cep_escola
  pagina_grafite.cell(row=10,column=5).value = f"{dia} DE {mes_digitado.upper()} DE {ano}"

  return dia,mes,ano,NF

arquivo_nce = load_workbook("MODELO NCE.xlsx")
pagina_nce = arquivo_nce["Matriz"]

arquivo_paper = load_workbook("MODELO PAPER.xlsx")
pagina_paper = arquivo_paper.active

arquivo_grafite = load_workbook("MODELO GRAFITE.xlsx")
pagina_grafite = arquivo_grafite.active

arquivo_controle = load_workbook("MODELO CONTROLE.xlsx")
pagina_controle = arquivo_controle.active

arquivo_escolas = load_workbook("escolas.xlsx")
pagina_escolas = arquivo_escolas.active
dia,mes,ano,NF = aplicar_informacoes(pagina_escolas,pagina_nce,pagina_paper,pagina_grafite)

arquivo_doador = load_workbook("NELSON.xlsx")
pagina_doador = arquivo_doador["Table 10"]

linha_ini_nce = 15
linha_ini_paper = 17
linha_ini_grafite = 12
linha_ini_controle = 3
linha_doadora = 3

precos_randomizados_por_valor = {}


while pagina_doador.cell(row=linha_doadora,column=2).value is not None:
  produto = pagina_doador.cell(row=linha_doadora,column=2).value
  un = pagina_doador.cell(row=linha_doadora,column=6).value
  qt = pagina_doador.cell(row=linha_doadora,column=7).value
  unit_nce = pagina_doador.cell(row=linha_doadora,column=8).value

  if unit_nce in precos_randomizados_por_valor:
    unit_paper,unit_grafite = precos_randomizados_por_valor[unit_nce]
  else:
   unit_paper = unit_nce * (1+ random.uniform(0.15,0.25))
   unit_grafite = unit_nce * (1+ random.uniform(0.15,0.25))
   while abs(unit_paper - unit_grafite)< 0.10:
     unit_grafite = unit_nce * (1+ random.uniform(0.15,0.25))
   precos_randomizados_por_valor[unit_nce] = unit_paper,unit_grafite


  pagina_nce.cell(row=linha_ini_nce,column=2).value = produto
  pagina_nce.cell(row=linha_ini_nce,column=3).value = un
  pagina_nce.cell(row=linha_ini_nce,column=4).value = qt
  pagina_nce.cell(row=linha_ini_nce,column=5).value = unit_nce


  pagina_paper.cell(row=linha_ini_paper,column=2).value = produto
  pagina_paper.cell(row=linha_ini_paper,column=3).value = un
  pagina_paper.cell(row=linha_ini_paper,column=4).value = qt
  pagina_paper.cell(row=linha_ini_paper,column=5).value = round(unit_paper,2)


  pagina_grafite.cell(row=linha_ini_grafite,column=2).value = produto
  pagina_grafite.cell(row=linha_ini_grafite,column=3).value = un
  pagina_grafite.cell(row=linha_ini_grafite,column=4).value = qt
  pagina_grafite.cell(row=linha_ini_grafite,column=5).value = round(unit_grafite,2)


  pagina_controle.cell(row=linha_ini_controle,column=2).value = produto
  pagina_controle.cell(row=linha_ini_controle,column=3).value = un
  pagina_controle.cell(row=linha_ini_controle,column=4).value = qt
  pagina_controle.cell(row=linha_ini_controle,column=5).value = unit_nce
  pagina_controle.cell(row=linha_ini_controle,column=8).value = round(unit_paper,2)
  pagina_controle.cell(row=linha_ini_controle,column=11).value = round(unit_grafite,2)


  linha_doadora +=1
  linha_ini_grafite +=1
  linha_ini_paper +=1
  linha_ini_nce +=1
  linha_ini_controle +=1

arquivo_nce.save(f"ORÇAMENTO NF{NF} {ano}-{mes}-{dia} NCE.xlsx")
arquivo_paper.save(f"ORÇAMENTO NF{NF} {ano}-{mes}-{dia} PAPER&CO.xlsx")
arquivo_grafite.save(f"ORÇAMENTO NF{NF} {ano}-{mes}-{dia} GRAFITE.xlsx")
arquivo_controle.save(f"MODELO DOC NF{NF}.xlsx")


Digite o CNPJ da escola que deseja bucar(sem . ou /, ex:123456789) : 1960308000141

 este cnpj corresponde a escola CONSELHO ESCOLAR COL EST NELSON RESENDE ALBUQUERQUE

Digite a data dos orçamentos (dia): 09
Digite a data dos orçamentos (mês): 07
Digite a data dos orçamentos (ano): 2025
Digite o número da nota fiscal correspondente : 10123
